<a href="https://colab.research.google.com/github/staller84/minhee/blob/master/baseline_%EB%B0%95%EB%AF%BC%ED%9D%AC_%EC%B5%9C%EC%A2%85%EC%A0%9C%EC%B6%9C%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

- Edit > Notebook settings > Hardward accelerators > GPU > SAVE
- Download the Friends dataset in EmotionLines website:
http://doraemon.iis.sinica.edu.tw/emotionlines/download.html
- Download the unlabeled json file.

# **Tutorials**

##### **Settings**

In [0]:
!pip install transformers --quiet # package installer for python tranformar packe 2가지가 반복적인 구조를 가진다.

In [0]:
import torch
from transformers import BertModel, BertTokenizer

In [0]:
pretrained_weights = 'bert-base-cased' #'bert-large-cased' 
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)
# pretrained 된걸 release해야 burt가 모두 구조적으로 동일하지만, 어떤 데이터를 얼마나 하는지 pretrain이 중요하다
# 이미 학습된 결과를 가져다 써야, 좋은 모델을 쓸 수 있다. 학습 체크 포인트. 

##### **Tokenization**

In [0]:
sentence = 'All the classes are provided.'

In [0]:
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']  
# 이게 특징적이다!!!! bert의 6개 토큰이 들어가면 6개의 임베딩이 출력된다. 
# 하지만 어떤 토큰을 쓴건가 결정? 이 CLS라는 특별한 토크능ㄹ 사용하게 된다. 그래서 실제로는 분절화된 토큰 뿐 아니라 
# 맨 앞의 CLS속성 이용하고 문장이 끝났다는 거르 의마흔 sep을 사용한다seperate의 약자
# 문장이 두개가 입력된다면 무저건 맨 앞ㅇ는 cls하나만 들어가고 txt a, b구분하는 sep가 두개 들어가낟 
# 문장이 하나일땐 scl,1ㄱ,sep1개지만 문장이 두개일땐 sep2는 2개 들어감
print(tokens)

['[CLS]', 'All', 'the', 'classes', 'are', 'provided', '.', '[SEP]']


In [0]:
ids = [tokenizer.convert_tokens_to_ids(tokens)]
print(ids) 
#토크나이저거 토큰을 id로 변환해서 컴퓨터에 숫자로 학습을 시켜야 한다. 

[[101, 1398, 1103, 3553, 1132, 2136, 119, 102]]


In [0]:
input_tensor = torch.tensor(ids)
print(input_tensor.data)
print(input_tensor.size()) #한문장 8개의 토큰 768개로 표현된다.

tensor([[ 101, 1398, 1103, 3553, 1132, 2136,  119,  102]])
torch.Size([1, 8])


##### **Model**

In [0]:
hidden_tensor = model(input_tensor)[0]
print(hidden_tensor.size()) 
#구글이 제공하는 텐서. id를 텐서로 변환해주는 과정이 필요하다. 값은 동일한데 (안에)
#  이거를 토치로 같은 텐서라 하더라도 파이토치와 텐서플러우의 텐서는 호환 안된다. 
# 적절한 것으로 변환하는게 필요하다
# 버트의 가장 작은 모델은 768다. 그래서 이거는 지켜줘야 한다.  큰 모델은 1024.. 이거는 뭔가 정해져 있는 값 같다. 

torch.Size([1, 8, 768])


In [0]:
logit = torch.nn.Linear(768, 3)(hidden_tensor) # base 모델은 768 large 모델은 : 1024
print(logit.size())
print(logit.data)

# 2를 3으로 바꾸기도 한다/차원임/
# 768을 3으로 클래스가 3개일때 사람이냐 강아지냐 고양이냐. task있을때 이 세개중 어디가 가장 크게 나오느냐를 
# 3으로 변환을 해주는데,,, 0.27ㅇ ㅣ가장 크면 첫번째 클래스를 선택하고, 그렂히 않은 경우 어디 선택하고~~~ 

torch.Size([1, 8, 3])
tensor([[[-0.2034, -0.1788, -0.0447],
         [-0.0730, -0.1098,  0.1147],
         [ 0.2424,  0.1843,  0.3871],
         [ 0.2827,  0.0196,  0.4151],
         [ 0.2748,  0.0970,  0.2352],
         [ 0.2982,  0.0414,  0.4705],
         [ 0.0234, -0.3860,  0.1145],
         [ 0.0084, -0.3636,  0.1091]]])


In [0]:
prediction = torch.nn.Softmax(dim=-1)(logit) 
print(prediction.size())
print(prediction.data)
  
# 음수를 0~1사이 확률값으로 조정해주겠다. softmax이용. 합이 1이 됨. 
# 사람이 해석 가능한 값으로 치환한 것 
# forward 방향으로 출력이 진행 됨. 예측된 결과가 얼마나 오차 큰지, 역으로 전파하며 back으로 가는데, 그거 트레이닝 과정 살펴봄
# toturial 어떻게 자연어 변환해서, 최종 클래스를 내느냐를 살펴보았따. 

torch.Size([1, 8, 3])
tensor([[[0.3128, 0.3206, 0.3666],
         [0.3154, 0.3040, 0.3805],
         [0.3227, 0.3044, 0.3729],
         [0.3436, 0.2641, 0.3923],
         [0.3574, 0.2991, 0.3435],
         [0.3377, 0.2612, 0.4011],
         [0.3624, 0.2406, 0.3970],
         [0.3577, 0.2466, 0.3956]]])


# **Emotion Recognition**

##### **Dataset**

In [0]:
import json

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('friends_' + dtype + '.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])



In [0]:
print(data)

{'train': {'speaker': ['Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Joey', 'Rachel', 'Joey', 'Rachel', 'Joey', 'Joey', 'Joey', 'Joey', 'Rachel', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Phoebe', 'Chandler', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Chandler', 'Phoebe', 'Joey', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Phoebe', 'Sergei', 'Phoebe', 'Sergei', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Joey', 'Ross', 'Phoebe', 'Phoeb

In [0]:
e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

In [0]:
print(e2i_dict)

{'sadness': 0, 'joy': 1, 'disgust': 2, 'non-neutral': 3, 'neutral': 4, 'fear': 5, 'anger': 6, 'surprise': 7}


In [0]:
print(i2e_dict)

{0: 'sadness', 1: 'joy', 2: 'disgust', 3: 'non-neutral', 4: 'neutral', 5: 'fear', 6: 'anger', 7: 'surprise'}


In [0]:
import collections
a = collections.Counter()
# print(a)
a.update(data['train']['emotion']) 
print(a)
print(len(a.keys()) ) # emtion : 8개 

a.update(data['test']['emotion']) 
# print(a)
a.update(data['dev']['emotion']) 
print(a) 
# dev : Counter({'neutral': 491, 'non-neutral': 214, 'surprise': 151, 'joy': 123, 'anger': 85, 'sadness': 62, 'fear': 29, 'disgust': 23})
# train Counter({'neutral': 4752, 'non-neutral': 2017, 'joy': 1283, 'surprise': 1220, 'anger': 513, 'sadness': 351, 'disgust': 240, 'fear': 185})
# test  Counter({'neutral': 1287, 'non-neutral': 541, 'joy': 304, 'surprise': 286, 'anger': 161, 'sadness': 85, 'disgust': 68, 'fear': 32})
# sum : Counter({'neutral': 6530, 'non-neutral': 2772, 'joy': 1710, 'surprise': 1657, 'anger': 759, 'sadness': 498, 'disgust': 331, 'fear': 246})

print(len(a.keys()) ) # emtion : 8개 

Counter({'neutral': 4752, 'non-neutral': 2017, 'joy': 1283, 'surprise': 1220, 'anger': 513, 'sadness': 351, 'disgust': 240, 'fear': 185})
8
Counter({'neutral': 6530, 'non-neutral': 2772, 'joy': 1710, 'surprise': 1657, 'anger': 759, 'sadness': 498, 'disgust': 331, 'fear': 246})
8


In [0]:
b = collections.Counter()
b.update(data['train']['utterance']) 
print(len(b.keys()) ) # utterance : 9,291개  

b.update(data['test']['utterance']) 
b.update(data['dev']['utterance']) 
print(b) 

print(len(b.keys()) ) # utterance : 12,645개  

9291
Counter({'Hey!': 141, 'Okay.': 113, 'What?': 99, 'Yeah.': 83, 'What?!': 53, 'Hi!': 53, 'Hey.': 49, 'Yeah!': 49, 'Hi.': 48, 'No.': 47, 'No!': 41, 'Yeah?': 37, 'Really?': 37, 'Oh.': 36, 'All right.': 24, 'Okay?': 24, 'Really?!': 23, 'Thank you.': 22, '!': 22, 'Oh!': 21, 'I know.': 20, 'Okay!': 20, 'Thanks.': 18, 'Oh my God!': 17, 'Huh?': 15, 'Why?': 14, 'Yes.': 14, 'Y\x92know?': 13, 'Uh-huh.': 13, 'Y\x92know what?': 13, 'I': 12, 'What are you doing?': 12, 'Hello.': 11, 'Ok.': 11, 'Oh my God.': 11, 'Yes!': 11, 'I know!': 11, 'I\x92m sorry.': 10, 'Come on!': 10, 'Fine!': 10, 'Ow!': 10, 'Here you go.': 9, 'Wow!': 9, 'Whoa!': 9, 'Fine.': 9, '.': 8, "I don't know.": 8, 'Hello?': 8, 'Ohh.': 8, 'Sorry.': 8, 'Why not?': 7, 'Thank you!': 7, 'Phoebe!': 7, 'Sure.': 7, 'Great!': 7, 'Good.': 7, 'Nothing.': 6, 'Bye.': 6, 'You are!': 6, 'So?': 6, 'Oh God!': 6, 'Ooh!': 6, 'What do you mean?': 6, 'Come on.': 6, 'Oh yeah!': 6, 'Uh-oh.': 5, 'Chandler!': 5, "Y'know?": 5, 'You do?': 5, 'Joey!': 5, 'Me t

In [0]:
c = collections.Counter()
c.update(data['train']['speaker']) 
print(len(c.keys()) )  # speaker : 264명

c.update(data['test']['speaker']) 
c.update(data['dev']['speaker']) 
print(c) 

print(len(c.keys()) ) # speaker : 309명

264
Counter({'Ross': 2166, 'Joey': 2163, 'Rachel': 2095, 'Phoebe': 1926, 'Monica': 1888, 'Chandler': 1861, 'Janice': 97, 'Carol': 63, 'Emily': 61, 'Tag': 60, 'All': 55, 'Mona': 48, 'Doug': 46, 'Frank': 43, 'Pete': 40, 'Joanna': 39, 'Mark': 38, 'Susan': 38, 'Richard': 33, 'Paul': 32, 'Woman': 30, 'David': 29, 'Gunther': 29, 'Mr. Treeger': 28, 'Danny': 28, 'Mr. Geller': 28, 'Phoebe Sr': 26, 'Mrs. Geller': 25, 'Julie': 25, 'Earl': 25, 'Elizabeth': 24, 'Director': 21, 'Eric': 21, 'Man': 21, 'Mike': 21, 'Dr. Green': 20, 'Gary': 19, 'Barry': 19, 'Guy': 19, 'Lydia': 18, 'Chip': 18, 'Kate': 16, 'Dina': 16, 'Nurse': 15, 'Jill': 15, 'Chloe': 15, 'Dana': 15, 'Charlie': 14, 'Alice': 14, 'Bonnie': 14, 'Leslie': 14, 'Janine': 14, 'Steve': 14, 'The Casting Director': 13, 'Mr. Tribbiani': 13, 'Dr. Long': 13, 'Joshua': 13, 'Kim': 13, 'Cassie': 13, 'Kristen': 13, 'Tour Guide': 13, 'Cliff': 12, 'Policeman': 12, 'Julio': 12, 'Katie': 12, 'Kathy': 12, 'Ben': 12, 'Russell': 12, 'Bob': 11, 'Duncan': 11, 'Mis

In [0]:
# 화자별 가장 많은 감정들 고려하기 (그사람의 성격!)
 
# data['train'].items()  # dict_items([('speaker', ['Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Cha
# list(data['train'].items())

# ex1 = data['train']['speaker']
# print(ex1)


# {키: 값 for 키, 값 in 딕셔너리 if 조건식}
# dict({키: 값 for 키, 값 in 딕셔너리 if 조건식})

# data1 = {key: value for key, value in data['train']['speaker'].items() if key == 'speaker'}
# print(data1 ) 
 

##### **Model**

In [0]:
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights) # 자연어를 어떻게 나눌지. 숫자 값으로 변환하고 
    self.bert_model = BertModel.from_pretrained(pretrained_weights)   #버트 모델 768로 나올텐데, 클래스가 우리가 가진 이모션 갯수가 8개
    self.linear = torch.nn.Linear(768, len(e2i_dict)) # 1024 버트 모델 중 하나를 예측하게 되는 것 

  def forward(self, utterance):
    tokens = self.bert_tokenizer.tokenize(utterance)
    tokens = ['[CLS]'] + tokens + ['[SEP]'] # (len)
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

In [0]:
print(input_tensor.shape)
print(hidden_tensor.shape)
print(logit.shape)



torch.Size([1, 8])
torch.Size([1, 8, 768])
torch.Size([1, 8, 3])


##### **Evaluation Metrics**

In [0]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)
  # 모델이 제대로 측정이 되었냐? 평가 메트릭을 어떻게 설정할 것인가? 얼마나 빠르게 학습 할건지, 몇번을 할건지.. 

##### **Hyper-parameters**

In [0]:
pretrained_weights = 'bert-base-cased' 
learning_rate = 1e-5
# learning_rate = 5e-4
n_epoch = 1  
#'bert-base-uncased' # base->large로 써도 된다.
# bert-base-uncased ->bert-large-uncased 로 바꿀 경우 위에서 1024로 바꿔야 한다. 
# bert-base-cased 로 써도 성능이 좋아질거다. 
# learning rate 바꿔도 좋아질거고, 에폭도 50, 100 이러게 하면 성능이 높아질거다. 이게 colab이라서 너무 오래 돌면.. 12시간 돌면 초기화 될꺼다. 
# 얼마나 학습을 빠르게 할꺼냐. 너무 빠르면, 성급한 일반화의 오류에,, 세세한 것에는 관심 못가질수도... 
# 적정수준 필요. 작게 잡는게 학습데이이터 많이 ㅣㄹ요하고 느리지만, 오랜시간 하습시 더 정확하다.
# 한 모델이 처음~끝, 한 에폭... 이 데이터를 여러번 읽음. 

In [0]:
print(range(len(data['train']['speaker'])))
print(range(len(data['test']['speaker'])))
print(range(len(data['dev']['speaker'])))

range(0, 10561)
range(0, 2764)
range(0, 1178)


##### **Training**

In [202]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda() 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# cuda() : mbdia GPU 생성하는데, 거기서 Mbida와 이게 호환 되도록 프로그램을 이용해 연결함. 이거를 cuda를 이용해 cpu쓰겠따는 뜻 
# LogSoftmax & NLLLoss  모델의 정확도. 맞췄냐 못맞췄냐..로스로 설정하겠다. 모델에 전달해서 모델 학습한다. 
for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  

  #  이거를 한 칸씩 들여 넣고, 몇번 했을때마다 모델 평가하는 형태로 바꾸면 조금 더 빠르게 할 수 이다. 
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))): # progress bar 이거는 노트북이라서 nb다. 그냥 tqdm notebook으로 하면 잘 나온다. 
    logit = model(data['dev']['utterance'][i_batch])
    _, max_idx = torch.max(logit, dim=-1) # torch.nn.Softmax(dim=-1)(logit) 
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  evaluate(pred_list, true_list) # print results

i_epoch: 0



precision:	 ['0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000']
recall:		 ['0.0000', '0.0000', '0.0000', '0.0000', '0.4168', '0.0000', '0.0000', '0.0000']
micro_f1: 0.416808


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


##### **Labeling**


In [200]:
from collections import OrderedDict

labeled = []
dialogs = json.loads(open('unlabeled.json').read())
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.json', 'w') as json_file:
  json.dump(labeled, json_file, indent='\t')

  # 학습하고,, unlabled된거를 레이블링 하는 작업이다. 

##### **Proposal**

- There is a class imbalance problem. (Use weighted cross-entropy etc.)

- Our model takes a single sentence. (Make it grasp its context as well.)

- Our model does not consider speaker information. (Make it consider the info.)

- Batch size is set as 1. (Increase the batch size.)

In [0]:
# 학습해서 labled를 내고, 파이썬 파일 수정한거 내고, 레포트는 각자 조사한거 제출하면 된다. 

# technical report :  버트 내부가 어덯게 하기때문에 이걸 사용했따던지.. 구체적으로 적어줄수록..좋을 것 같다. 
# 자료의 구성과 흐름... 성의... 기승전결.. 얼마나 잘 이뤄졌느냐. 이문제가 머가 왜 이걸 썻고, 그래서 어떻게 해결했꼬, 어떤 문제가 있다는 형태의 
# 논문형태로 써주면 좋을 것 같습니다. 

# proposal 은 참고하라고 적어 주신거고. 알아서 개인적으로 의견 넣으면 된다. 

# 감정이 굉장히 biased 되어 있는데, 이거를 어떻게 균형잡히게 하면 될지, 
# crossendtrophyloss(weight....) 이렇게 넣으면 더 늘어난다. 
# 문장이 아니라 문맥을
# 스피처.. 배치 사이즈를 늘려서 여러개를 종합해서,, 
# 로스를 개선학ㅆ다는 거라서 그거를 키우면 학습이 잘될꺼다. 개선점들을 바꿔봐라 
# 화자가 누구인지를 넣으면 성능이 더 좋아질 수 있다. 

# jay alammar 의 딥러닝... 이거 참고할 것